# Credit Risk Resampling Techniques

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [ ]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [ ]:
# Load the data
file_path = Path('resource/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

# Split the Data into Training and Testing

In [ ]:


# Create our features
# YOUR CODE HERE
X = pd.get_dummies(df.drop('loan_status', axis = 1))
# Create our target
# YOUR CODE HERE
y = df[ "loan_status" ]

In [ ]:
X.describe()

In [ ]:
# Check the balance of our target values
y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

X_train_scaled =  X_scaler.transform(X_train)

X_test_scaled =   X_scaler.transform(X_test)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [ ]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
from imblearn.over_sampling import RandomOverSampler

Rsamp = RandomOverSampler(random_state=1)
X_resampled, y_resampled = Rsamp.fit_resample( X_train_scaled, y_train )
Counter( y_resampled )

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression
O_model = LogisticRegression(solver='lbfgs', random_state = 1)
O_model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score
y_prediction = O_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_prediction)

In [ ]:
from sklearn.metrics import confusion_matrix
# Display the confusion matrix
# YOUR CODE HERE


confusion_matrix(y_test, y_prediction)

In [ ]:
from imblearn.metrics import classification_report_imbalanced
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_prediction))

### SMOTE Oversampling

In [ ]:
from imblearn.over_sampling import SMOTE
# Resample the training data with SMOTE
# YOUR CODE HERE

Strain = SMOTE(random_state=1, sampling_strategy = 1.0)
X_resampled, y_resampled = Strain.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
Strain_model = LogisticRegression(solver='lbfgs', random_state = 1)
Strain_model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
Strain_prediction = Strain_model.predict(X_test_scaled)

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, predictions)

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, Strain_predictions))

# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [ ]:
from imblearn.under_sampling import ClusterCentroids
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
# YOUR CODE HERE

CentroidSample = ClusterCentroids(random_state = 1)
X_resampled, y_resampled = CentroidSample.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
Clusters = LogisticRegression(solver = 'lbfgs', random_state = 1)
Clusters.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

Cluster_predictions = cluster_model.predict(X_test_scaled)

balanced_accuracy_score(y_test, Cluster_predictions)

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, Cluster_predictions)

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, Cluster_predictions))

# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [ ]:
from imblearn.combine import SMOTEENN
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
# YOUR CODE HERE

STeen = SMOTEENN(random_state  = 1)
X_resampled, y_resampled = STeen.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

SteenModel = LogisticRegression(solver = 'lbfgs', random_state = 1)
SteenModel.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

SteenPrediction = SteenModel.predict(X_test_scaled)

balanced_accuracy_score(y_test, SteenPrediction)

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, SteenPrediction)

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, SteenPrediction))